# Lab on Stochastic Linear Bandits :

We provide the environment to run a standard linear bandit experiment. The objective of this lab session is to understand how to implement LinUCB, the algorithm seen in class and its variant LinTS. We shall see that in practice there are some shortcomings in the implementation to make it efficient so we will guide you to obtain a working version.

Questions are inline in the notebook and some reserved space are allocated for answers, but feel free to add cells for remarks and run your own experiments to test hypotheses you may have.


In [1]:
import numpy as np
from scipy.stats import bernoulli
from math import log

import random
import sys
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
colors = sns.color_palette('colorblind')

In [2]:
def plot_regret(regrets, logscale=False, lb=None,q=10):
    """
    regrets must be a dict {'agent_id':regret_table}
    """

    reg_plot = plt.figure()
    #compute useful stats
#     regret_stats = {}
    for i, agent_id in enumerate(regrets.keys()):
        data = regrets[agent_id]
        N, T = data.shape
        cumdata = np.cumsum(data, axis=1) # cumulative regret

        mean_reg = np.mean(cumdata, axis=0)
        q_reg = np.percentile(cumdata, q, axis=0)
        Q_reg = np.percentile(cumdata, 100-q, axis=0)

#         regret_stats[agent_id] = np.array(mean_reg, q_reg, Q_reg)

        plt.plot(np.arange(T), mean_reg, color=colors[i], label=agent_id)
        plt.fill_between(np.arange(T), q_reg, Q_reg, color=colors[i], alpha=0.2)

    if logscale:
        plt.xscale('log')
        plt.xlim(left=100)

    if lb is not None:
        plt.plot(np.arange(T), lb, color='black', marker='*', markevery=int(T/10))

    plt.xlabel('time steps')
    plt.ylabel('Cumulative Regret')
    plt.legend()
    reg_plot.show()

# Environment Class

The environment class allows to create 3 types of linear bandit problems:
* 'fixed': normally requires a fixed_actions input (otherwise randomly generated at start) which is kept all along the game;
* 'iid': at each round, the environment samples K actions at random on the sphere.

For each of these types of game, the class is used to generate the action sets at each round and the reward for a chosen action (chosen by an Agent, see the "Play!" section for the details of the interaction).

### Action generators
Please implement a function that generates K actions in dimension d. You may want to check the lecture slides to see whether some conditions should be respected.

In the report, explain and justify your choice.

In [3]:
def ActionsGenerator(K,d, mean=None):
    """
    K: int -- number of action vectors to be generated
    d : int -- dimension of the action space
    returns : an array of K vectors uniformly sampled on the unit sphere in R^d
    """
    # Random vector with mean = 0
    rand_vector = np.random.normal(size=(K, d))
    
    # Normalize each vector to have unit norm
    norms = np.linalg.norm(rand_vector, axis=1, keepdims=True)
    unit_vectors = rand_vector / norms

    # If a mean vector is provided, add it to the unit_vector
    if mean is not None:
        unit_vectors += mean
        
        # Normalize again to ensure they remain on the unit sphere
        norms = np.linalg.norm(unit_vectors, axis=1, keepdims=True)
        unit_vectors = unit_vectors / norms

    return unit_vectors

### Linear Bandit environment

The following class is your environment: it generates an action set of K vectors at each round and returns the (random) reward given an action.
You can see how it is used in the experiment function further below.

In [181]:
class LinearBandit:

    def __init__(self, theta, K, var=1., fixed_actions=None):
      """
      theta: d-dimensional vector (bounded) representing the hidden parameter
      K: number of actions per round (random action vectors generated each time)
      pb_type: string in 'fixed', 'iid', 'nsr' (please ignore NotSoRandom)
      """
      self.d = np.size(theta)
      self.theta = theta
      self.K = K
      self.var = var
      self.current_action_set = np.zeros(self.d)
      self.fixed_actions = fixed_actions

    def get_action_set(self):
      """
      Generates a set of vectors in dimension self.d. Use your ActionsGenerator
      Alternatively, the set of actions is fixed a priori (given as input).
      Implement a condition to return the fixed set when one is given
      """
      if self.fixed_actions is not None:
        return self.fixed_actions
      else: 
        self.current_action_set = ActionsGenerator(self.K, self.d)
        return ActionsGenerator(self.K, self.d)


    def get_reward(self, action):
      """ sample reward given action and the model of this bandit environment
      action: d-dimensional vector (action chosen by the learner)
      """
      # self.current_action_set = action
      mean = np.dot(action, self.theta)
      return np.random.normal(mean, scale=self.var)

    def get_means(self):
      # print(f'current action_set shape = {self.current_action_set.shape}')
      # print(f'theta shape = {self.theta.shape}')
      return np.dot(self.current_action_set, self.theta)
    
    def __repr__(self):
      str = 'Object Environnement'
      str += f'\n d = {self.d}'
      str += f'\n Theta = {self.theta}, shape = {self.theta.shape}'
      str += f'\n Current_action_set = {self.current_action_set}, shape = {self.current_action_set.shape}'
      
      return str

# Play !
The function play runs one path of regret for one agent. The function experiment runs all agents several (Nmc) times and returns all the logged data. Feel free to check the inputs and outputs required when you decide on the implementation of your own agents.

In [259]:
def play(environment, agent, Nmc, T, pseudo_regret=True):
    """
    Play one Nmc trajectories over a horizon T for the specified agent.
    Return the agent's name (sring) and the collected data in an nd-array.
    """

    data = np.zeros((Nmc, T))

    for n in range(Nmc):
        agent.reset()
        for t in range(T):
            action_set = environment.get_action_set()
            # print(f'action set shape = {action_set.shape}')
            # print(f'env = {env}')
            action = agent.get_action(action_set)
            # print(f'action from get action = {action.shape}')
            reward = environment.get_reward(action)
            agent.receive_reward(action,reward)

            # compute instant (pseudo) regret
            means = environment.get_means()
            best_reward = np.max(means)
            if pseudo_regret:
              # pseudo-regret removes some of the noise and corresponds to the metric studied in class
              data[n,t] = best_reward - np.dot(environment.theta,action)
            else:
              data[n,t]= best_reward - reward # this can be negative due to the noise, but on average it's positive

    return agent.name(), data


def experiment(environment, agents, Nmc, T,pseudo_regret=True):
    """
    Play Nmc trajectories for all agents over a horizon T. Store all the data in a dictionary.
    """

    all_data = {}

    for agent in agents:
        agent_id, regrets = play(environment, agent,Nmc, T,pseudo_regret)

        all_data[agent_id] = regrets

    return all_data

# Linear Bandit Agents






## LinUCB : Implementing optimism in $R^d$

As seen in class, the actions are now vectors in $R^d$, representing contextual features, and the environment is assumed to generate rewards according to some hidden linear function $f_\theta(a) = a^\top \theta$. The goal of the learner is thus to estimate $\theta$ while keeping a measure of the uncertainty in all the directions of the feature space.

* **Baseline: Implementation of LinEpsilonGreedy** In the next cell, we implemented a LinUniform Agent that returns one of the action vectors of the action set, chosen uniformly at random. Please implement a `LinEpsilonGreedy` agent and test it against `Greedy` ($\epsilon=0$) on the 2 proposed  environments (iid and fixed actions). What do you notice? Is $\epsilon$-Greedy a good baseline to test algorithms?


* **Implementation of LinUCB**: you need to compute UCBs for each arm of the current action set received from the environment, but this time the exploration bonus depends on the history of taken actions and received rewards (see course material).

* **Efficiency of the matrix inversion step**: One key step is to invert the covariance matrix in order to compute the elliptical norm of each available action. Remark however that at round $t+1$, the new covariance matrix is very similar to the previous one at rount $t$... Can you think of a way to optimize this step by simply updating the old one ? Please implement this improvement as an additional option to your `LinUCB` agent so you can compare runtimes from bandit problems in dimension $d=2,8,16,32,64$. Plot the result of the compared runtimes. In your report,  discuss the computational complexity of this step and the resulting improvement, give your new update formula, and report the plot of compared runtimes.

### Uniform random policy

In [204]:
class LinUniform():
  def __init__(self):
    # self.d = d
    # self.lambda_reg = lambda_reg
    # self.reset()
    pass
  
  def get_action(self, arms):
    K, _ = arms.shape
    return arms[np.random.choice(K)]

  def receive_reward(self, chosen_arm, reward):
    # chosen_arm = chosen_arm.reshape(-1, 1)

    # self.action += chosen_arm.T@chosen_arm
    # self.invcov = np.linalg.pinv(self.cov + self.action) 


    # #update b_t
    # self.b_t += reward * chosen_arm

    # self.hat_theta = self.invcov@ self.b_t # update the least square estimate
    pass
  
  def reset(self):
    # self.hat_theta = np.zeros(self.d).reshape(-1, 1)
    # self.cov = self.lambda_reg * np.identity(self.d)
    # self.invcov = np.identity(self.d)
    # self.b_t = np.zeros(self.d).reshape(-1, 1)
    # self.action = np.zeros(self.d).reshape(-1, 1)
    pass
  def name(self):
    return 'Unif'

### Lin-$\epsilon$-Greedy policy:

In [205]:

from numpy.linalg import pinv
from time import time

class LinEpsilonGreedy:
  def __init__(self, d,lambda_reg, eps=0.1, other_option=None):
    self.eps = eps # exploration probability
    self.d = d
    self.lambda_reg = lambda_reg
    self.reset()
    self.other_option = other_option
    #use other inputs if needed

  def reset(self):
    """
    This function should reset all estimators and counts.
    It is used between independent experiments (see 'Play!' above)
    """
    self.t = 0
    self.hat_theta = np.zeros(self.d).reshape(-1, 1)
    self.cov = self.lambda_reg * np.identity(self.d)
    self.invcov = np.identity(self.d)
    self.b_t = np.zeros(self.d).reshape(-1, 1)
    self.action = np.zeros(self.d).reshape(-1, 1)
    

  def greedy_or_not(self, K, arms):
    """This function picks the greedy arm with a probability (p = 1 - epsilon)
    It tries a random arm with a probability (p = epsilon)"""

    threshold = np.random.rand() # Pick random numbers between 0 and 1

    if self.eps < threshold: # Greedy strategy, pick the arm that provides the best rewards
      expected_reward = arms@self.hat_theta
      index = np.argmax(expected_reward) # changer ça
      return arms[index, :]
    else : # Explore strategy, pick a random arm
      K, _ = arms.shape
      return arms[np.random.choice(K)]


  def get_action(self, arms):
    K = arms.shape[0] # number of arms

    action = self.greedy_or_not(K, arms)
    return action
  
  def sherman_morrison(self, P_inv, a):
      denominator = (1 + a.T@P_inv@a)
      numerator = (P_inv@a)@(a.T@P_inv)
      return numerator/denominator
  
  def receive_reward(self, chosen_arm, reward):
    """
    update the internal quantities required to estimate the parameter theta using least squares
    """
    #update inverse covariance matrix
    chosen_arm = chosen_arm.reshape(-1, 1)
    if self.other_option is not None :
      if self.t == 0:
        self.invcov = np.linalg.pinv(self.cov) 
        # self.invcov -= np.copy(self.sherman_morrison(self.invcov, chosen_arm))
      else :
        self.invcov -= self.sherman_morrison(self.invcov, chosen_arm)
    else :
      self.action += chosen_arm.T@chosen_arm
      self.invcov = np.linalg.pinv(self.cov + self.action) 

    #update b_t
    self.b_t += reward * chosen_arm

    self.hat_theta = self.invcov@self.b_t # update the least square estimate
    self.t += 1

  def name(self):
    return 'LinEGreedy('+str(self.eps)+')'
  
  def __repr__(self):
    str = f'Object LinEpsilonGreedy'
    str += f'\nTime step = {self.t}'
    str += f'\nTheta Hat = {self.hat_theta}, shape = {self.hat_theta.shape}'
    str += f'\nCov matrix = {self.cov}, shape = {self.cov.shape}'
    
    return str


### Default setting

Quick test: I propose some simple settings to run a quick test.
For your report, feel free to change these and explain / justify your choices.
Please report the resulting plot and your conclusions on Lin-E-Greedy.

In [206]:
d = 15  # dimension
K = 30  # number of arms

# parametor vector \theta, normalized :
theta = ActionsGenerator(1,d).reshape(-1) # another way of getting a 'random vector' (depends on your implementation)
# theta = np.array([0.45, 0.5, 0.5])
theta /= np.linalg.norm(theta)
T = 3  # Finite Horizon
N = 2  # Monte Carlo simulations

delta = 0.1 # could be set directly in the algorithms
sigma = 1.

#choice of percentile display
q = 10


In [207]:
env = LinearBandit(theta, K, var=sigma**2)

print(env)

Object Environnement
 d = 15
 Theta = [ 0.17104176 -0.12780983  0.05430021 -0.37973898 -0.23032588  0.22736794
 -0.55385881 -0.25058229  0.08835728  0.20628349  0.06120807  0.01363562
 -0.11288705 -0.43321885 -0.27972771], shape = (15,)
 Current_action_set = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape = (15,)


In [208]:
# policies

uniform = LinUniform()
e_greedy = LinEpsilonGreedy(d, lambda_reg=1., eps=0.1, other_option=None)
greedy = LinEpsilonGreedy(d, lambda_reg=1., eps=0., other_option=None)

In [209]:
exp = experiment(env, [greedy, e_greedy, uniform], Nmc=N, T=T,pseudo_regret=True)
plot_regret(exp)

action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)
action from get action = (15,)


/tmp/ipykernel_8825/3942893175.py:33: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  reg_plot.show()


Is Lin-E-Greedy a strong baseline? Is it hard to beat?

The epsilon greedy algorithm was tested on a simple problem where $\theta$ is defined as a random vector between 0 and 1 and all the actions are generated randomly on a unit sphere. This algorithm is naive as it keeps on exploring when all states are visited, which causes the regret to grow when $\epsilon > 0$. To increase performance, we would expect to use an algorithm that explores less over time. Therefore, despite its simplicity, the epsilon greedy algorithm is a good baseline to compare with more sophisticated algorithms. 

## Matrix inversion complexity discussion

The matrix inversion complexity is $O(d^3)$. Therefore, as d increases, this matrix inversion becomes the bottleneck of the epsilon greedy algorithm. To improve performance, we could use the sherman morrison method that has $O(d^2)$ complexity.

In [189]:
import time

d_vector = np.array([3, 10, 50, 100])
time_vector = np.zeros((2, len(d_vector)))

# Computational time comparison of both matrix inversion techniques
for i, d_val in enumerate(d_vector):
    theta = ActionsGenerator(1,d_val).reshape(-1)
    env_time = LinearBandit(theta, K, var=sigma**2)
    
    start = time.time()
    greedy_1 = LinEpsilonGreedy(d_val, lambda_reg=1., eps=0.1, other_option = True)
    exp1 = experiment(env_time, [greedy_1], Nmc=N, T=T,pseudo_regret=True)
    end = time.time() - start
    plot_regret(exp1)
    time_vector[0, i] = end

    start_2 = time.time()
    greedy_2 = LinEpsilonGreedy(d_val, lambda_reg=1., eps=0.1)
    exp2 = experiment(env_time, [greedy_2], Nmc=N, T=T,pseudo_regret=True)
    end_2 = time.time() - start_2
    time_vector[1, i] = end_2
    plot_regret(exp2)

/tmp/ipykernel_8825/3942893175.py:33: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  reg_plot.show()


In [190]:
plt.figure()
plt.semilogy(d_vector, time_vector[0,:], label = "Sherman Morrison formula")
plt.semilogy(d_vector, time_vector[1,:], label = "Classical matrix inversion")
plt.xlabel("d dimension")
plt.ylabel("Computational time - s")
plt.legend()
plt.title("Epsilon greedy computational time comparison")

Text(0.5, 1.0, 'Epsilon greedy computational time comparison')

## Lin-UCB: The optimistic way

**Implement LinUCB** as seen in class and test it against the baselines implemented above.

If you are happy with the result, move to the last part below.


In [252]:
class LinUCB:
    
  # constructeur - init paramètres :
  # - d = dimension
  # - lambda_reg = regularization parameter
  # - ??
  def __init__(self, d, delta, sigma=sigma, lambda_reg=1., other_option=None, T=1000):
      
    self.d = d
    self.delta = delta
    self.sigma = sigma
    self.lambda_reg = lambda_reg
    self.T = T
    
    self.other_option = other_option   
    self.reset()

  # reset method - used in the play function loop
  def reset(self):
    """
    This function should reset all estimators and counts.
    It is used between independent experiments (see 'Play!' above)
    """
    self.t = 0
    self.theta_hat = np.zeros(self.d).reshape(-1, 1)
    self.cov = self.lambda_reg * np.identity(self.d)
    self.invcov = np.identity(self.d)
    self.history_design_matrix = np.zeros((d,d)) # sum over s of x_s . x_s^{T}
    self.history_rx = np.zeros(d) # sum over s of r_s . x_s

  def _sherman_morrison(self, P_inv, a):
      denominator = (1 + a.T@P_inv@a)
      numerator = (P_inv@a)@(a.T@P_inv)
      return numerator/denominator
    
  def _compute_ucbs(self, K, arms):
          
      # calculate UCB bounds
      ucbs = np.zeros(K)
      for k in range(K):
          action = arms[k]
          beta = self.sigma * np.sqrt(2*np.log(self.T) + self.d * np.log(1+self.t / (self.d * self.lambda_reg)) /
                                 + np.sqrt(self.lambda_reg))
          norm_action = np.sqrt(np.matmul(action.T, np.matmul(self.invcov, action)))
          # print(f'action shape = {action.shape}')
          # print(f'self.theta_hat reshape = {self.theta_hat.reshape(-1,).shape}')
          ucbs[k] = np.dot(action, self.theta_hat.reshape(-1,)) + norm_action * beta
          
      chosen_arm_id = np.argmax(ucbs)
      chosen_action = arms[chosen_arm_id].reshape(-1,)
      
      return chosen_arm_id, chosen_action

  # choose the arm per policy
  def get_action(self, arms):
      
    K = arms.shape[0] # number of arms
    chosen_arm_id, chosen_action = self._compute_ucbs(K, arms)
    # print(chosen_action)
    return chosen_action
  
  # update internal parameters
  def receive_reward(self, chosen_arm, reward):
    """
    update the internal quantities required to estimate the parameter theta using least squares
    """
    
    # update sum of xx.T
    self.history_design_matrix += np.matmul(chosen_arm.T, chosen_arm)
    # update sum of rx
    # print(f'reward = {reward}')
    # print(f'history_rx = {self.history_rx}.shape')
    self.history_rx += reward * chosen_arm
    # update cov matrix
    self.cov = self.lambda_reg * np.identity(self.d) + self.history_design_matrix # B_t_lambda
    
    #update inverse covariance matrix, possibly with sherman-morrisson
    # chosen_arm = chosen_arm.reshape(-1, 1)
    
    # if self.other_option is not None :
    #   if self.t == 0:
    #     self.invcov = np.linalg.pinv(self.cov) 
    #   else :
    #     self.invcov -= self._sherman_morrison(self.invcov, chosen_arm)
    # else :
    #   self.action += chosen_arm.T@chosen_arm
    #   self.invcov = np.linalg.pinv(self.cov + self.action)
    self.invcov =  np.linalg.pinv(self.cov)
      
    # update theta estimator
    self.theta_hat = np.matmul(self.invcov, self.history_rx)
    
    # update histories
    self.t += 1

  def name(self):
    return 'LinUCB'
  
  def __repr__(self):
    str = f'Object LinUCB'
    str += f'\nTime step = {self.t}'
    str += f'\nTheta Hat = {self.theta_hat}, shape = {self.theta_hat.shape}'
    str += f'\nCov matrix shape = {self.cov.shape}'
    
    return str

### Test against baselines

In [260]:
# policies

linucb = LinUCB(d, delta, sigma=sigma, lambda_reg=1.)
uniform = LinUniform()
e_greedy = LinEpsilonGreedy(d, lambda_reg=1., eps=0.1)
greedy = LinEpsilonGreedy(d, lambda_reg=1., eps=0.)

In [261]:
theta = ActionsGenerator(1,d).reshape(-1) # another way of getting a 'random vector' (depends on your implementation)
theta /= np.linalg.norm(theta)

iid_env = LinearBandit(theta, K, var=sigma**2)

In [262]:
iid_env

Object Environnement
 d = 15
 Theta = [-0.15487514 -0.12260666 -0.1005312  -0.00389548 -0.15721351 -0.0817665
 -0.55792103 -0.2544118  -0.0751942   0.22259529  0.08729594  0.13919493
 -0.10002854  0.66243962  0.11147161], shape = (15,)
 Current_action_set = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape = (15,)

In [263]:
linucb

Object LinUCB
Time step = 0
Theta Hat = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape = (15, 1)
Cov matrix shape = (15, 15)

In [264]:
# e_greedy

In [ ]:
linucb_vs_greedy = experiment(iid_env, [linucb, greedy, e_greedy], Nmc=N, T=T,pseudo_regret=True) # greedy, e_greedy, 
plot_regret(linucb_vs_greedy)

/tmp/ipykernel_8825/3942893175.py:33: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  reg_plot.show()


## LinTS : Taking the Bayesian way

Thompson Sampling is a popular bayesian alternative to the standard optimistic bandit algorithms (see Chapter 36 of Bandit Algorithms). The key idea is to rely on Bayesian *samples* to get a proxy for the hidden parameter $\theta$ of the problem instead of building high-probability confidence regions.

* **Posterior derivation**: Let us place a Gaussian prior with mean $\mathbf{0}$ and covariance $\lambda I$ on $\theta$. Given actions $A_1,\ldots,A_t$ and rewards $Y_1,\ldots,Y_t$, Can you compute the expression of the posterior at the beginning of round $t+1$ ?

In your report, write the distribution of the posterior as a function of the prior and the observed data. No need to report your full derivation if you are lacking space.


* **Implementation of LinTS**. Please implement Linear Thompson Sampling using the formula you derived above.

In [ ]:
# class LinTS ... (use you LinUCB template)


### Comparison and report

Compare LinUCB, LinTS and LinEGreedy on a problem of your choice. In your report, explain your choice of problem and report the plot of your experiment as well as a few sentences of comment: Is there a clear 'winner'?

#Bonus

In this bonus part, we explore the role of the action sets on the performance of our algorithms.

In class we said that action sets can be 'arbitrary'. This means that, in principle, they do not have to follow a distribution, they do not have to be random either.

What happens if the action set is fixed?

We propose an alternative 'play' function that fixes the action set:

In [ ]:
def play_fixed(environment, agent, Nmc, T, actions=None, pseudo_regret=True):
    """
    Play one Nmc trajectories over a horizon T for the specified agent.
    Return the agent's name (sring) and the collected data in an nd-array.
    actions: a fixed action set. Default is set to be the canonical basis.
    """

    data = np.zeros((Nmc, T))

    for n in range(Nmc):
        agent.reset()
        for t in range(T):
            # action_set = environment.get_action_set() -> We no longer call on your ActionsGenerator function
            action_set = np.copy(actions) # the actions given as input
            action = agent.get_action(action_set)
            reward = environment.get_reward(action)
            agent.receive_reward(action,reward)

            # compute instant (pseudo) regret
            means = environment.get_means()
            best_reward = np.max(means)
            if pseudo_regret:
              data[n,t] = best_reward - np.dot(environment.theta,action)
            else:
              data[n,t]= best_reward - reward # this can be negative due to the noise, but on average it's positive

    return agent.name(), data

In [ ]:
def experiment_fixed(environment, agents, Nmc, T, actions=None, pseudo_regret=True):
    """
    Play Nmc trajectories for all agents over a horizon T. Store all the data in a dictionary.
    """

    all_data = {}
    if actions is None:
      actions = np.ActionsGenerator(K,d) #call it once!
    print(actions)

    for agent in agents:
        agent_id, regrets = play_fixed(environment, agent, Nmc, T, actions=actions, pseudo_regret=pseudo_regret)

        all_data[agent_id] = regrets

    return all_data

Now the actions are fixed, so we could actually use UCB to address the problem: after all, it is just a K-armed bandit, but with structure.

**When is LinUCB better than UCB?**

In [ ]:
class UCB:
  def __init__(self, K, var):
      self.K = K
      self.var = var
      self.count_actions = np.zeros(self.K)
      self.count_rewards = np.zeros(self.K)
      self.t = 0

  def get_action(self,action_set):
      if self.t < self.K:
        action = self.t
      else:
        empirical_means = self.count_rewards / self.count_actions
        ucbs = np.sqrt(6 * self.var * np.log(self.t) / self.count_actions) # 6 could be replaced by a 2, try it out :)
        action = np.argmax(empirical_means + ucbs)

      self.t += 1
      self.count_actions[action] += 1
      self.current_action = action #need to remember the *index* of the action now
      return action_set[action,:]

  def receive_reward(self, action, reward):
      self.count_rewards[self.current_action] += reward

  def reset(self):
      self.count_actions = np.zeros(self.K)
      self.count_rewards = np.zeros(self.K)
      self.t = 0

  def name(self):
      return 'UCB('+str(self.var)+')'

In [ ]:
d=3
K=7 # same as before, more actions than dimension

env = LinearBandit(theta, K)
ucb = UCB(K,var=1.)
linucb = LinUCB(d, delta=0.01, lambda_reg=1., sigma=1. )
reg_fixed_actions = experiment_fixed(env, [ucb, linucb], Nmc=10, T=200, actions=None, pseudo_regret=True)

In [ ]:
plot_regret(reg_fixed_actions)

In general, LinUCB is better in such problem because it shares information across actions.

Now, what if K=d and the fixed action set is exactly the canonical basis (K=d independent actions)

In [ ]:
d=7
K=d
# theta = np.random.multivariate_normal(np.zeros(d),np.eye(d))
theta = np.linspace(0.1,1,num=d) # just d actions in increasing value order
#theta = your choice of parameter
theta /= np.linalg.norm(theta) #optional if you set theta with bounded norm :)

env = LinearBandit(theta, d, fixed_actions=np.eye(d))
ucb = UCB(d,var=1.)
linucb = LinUCB(d, delta=0.01, lambda_reg=1., sigma=1. )
reg_fixed_actions = experiment_fixed(env, [ucb, linucb], Nmc=10, T=200, actions=np.eye(d), pseudo_regret=True)

In [ ]:
plot_regret(reg_fixed_actions)

The two algorithms should be roughly on par. In fact, it is possible to refine UCB to get slightly better performance (by tightening a bit the upper confidence bounds).

In "[The End of Optimism](https://arxiv.org/pdf/1610.04491)", Lattimore et al. show that for a certain type of action set, one can show that UCB has **linear regret**. A famous 'counter-example' (bad action set) is given in Section 8 therein.

**Exercise**: find the action set (3 actions in dimension 2) and run the experiment above, what do you see? In (the appendix of) your report, please report your figures and conclusions.

The problem of finding a good algorithm for the 'arbitrary' setting and the 'fixed-design' setting was open for a long time but recent papers (e.g. [Kirschner et al.,2021](https://arxiv.org/abs/2011.05944)) have now proposed solutions.